In [ ]:
%matplotlib inline

In [ ]:


# # Tutorial notebook for predicting a missing component in a regularly varying random vector -- work in progress


# 
# import os
# os.getcwd()
# os.chdir("../")
# os.getcwd()
# 

# 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import MLExtreme as mlx
# 
#
# Norm function: L2 norm. change `norm_order` to 1 or inf for variants.
# NB: inf is only applicable to the linear transform below, not the nonlinear
# transform
#
norm_order = 2


def norm_func(x):
    return np.linalg.norm(x, ord=norm_order, axis=1)


# 
# Data generation
np.random.seed(42)
n = int(4 * 10**4)
Dim = 2
k = 2
alpha = 2
# Mu0 =  np.array([[0.1,  0.7], [0.6,  0.4], [0.9, 0.1] ])
Mu0 = np.array([[0.1, 0.7, 0.2], [0.2, 0.1, 0.7]])
wei0 = np.ones(2)
Mu, wei = mlx.normalize_param_dirimix(Mu0, wei0)
lnu = np.log(10 / np.min(Mu, axis=1))
# define  adversarial bulk angular measure, see function gen_rv_dirimix
Mu_bulk = np.array([[0.7, 0.1, 0.2], [0.1, 0.2, 0.7]])

XZ = mlx.gen_rv_dirimix(alpha, Mu, wei, lnu, scale_weight_noise=1,
                        index_weight_noise=3, Mu_bulk=Mu_bulk,
                        size=n)
X = XZ[:, :-1]
Z = XZ[:, -1]


# ## plot limit angular measure and bulk angular measure
mlx.plot_pdf_dirimix_3D(Mu, wei, lnu, n_points=100)
mlx.plot_pdf_dirimix_3D(Mu_bulk, wei, lnu, n_points=100)

## Tranforming the target :

### Option 1: learn a prediction model for a linear tranform  of the target,
### y =  Target / ||X||.
### appropriate when  Target / ||X|| is bounded or at least not clearly
### heavy tailed

In [ ]:
y1 = mlx.transform_target_lin(Z, X, norm_func)

# test
Z1 = mlx.inv_transform_target_lin(y1, X, norm_func)
np.sum((Z1 - Z)**2)  # OK

### Option 2:  Nonlinear transformation :
### y = z / (||(x,z) ||) where z is the original target,
### and x the covariate
Appropriate when z/||x||  may not be considered bounded.
if ||.|| is the L_q norm, the inverse transform is:
 z =  y/ (1 - y**q)**(1/q)  * || x ||
NB: the infinite norm is not appropriate here, only L_q norms, q in [1, inf)

In [ ]:
y2 = mlx.transform_target_nonlin(Z, X, 2)

# test
Z2 = mlx.inv_transform_target_nonlin(y2, X, 2)
np.sum((Z2 - Z)**2) # OK

# boundedness assumption for y1?
plt.hist(y1)
plt.show()
# warning here: Some of the (comparatively) largest values of y1 may cause
# instability.

# what about y2?
plt.hist(y2)
plt.show()
# as expected y2 it is strictly comprised in [0,1]

# Vizualisation of  y1 / y2 versus x. 
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=y1, cmap='gray', alpha=0.5)
axes[0].set_title('Scatter Plot with y1')
axes[0].set_xlabel('X1')
axes[0].set_ylabel('X2')
scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=y2, cmap='gray', alpha=0.5)
axes[1].set_title('Scatter Plot with y2')
axes[1].set_xlabel('X1')
axes[1].set_ylabel('X2')
fig.colorbar(scatter1, ax=axes[0])
fig.colorbar(scatter2, ax=axes[1])
plt.tight_layout()
plt.show()
# similar pattern 

### Train/test split: here (simulated data) half/half

In [ ]:
# for y1: Splitting the data into training and test sets
split = 0.5
ntrain = n * (1-split)
ntest = n * split 
X_train, X_test, y_train1, y_test1 = train_test_split(X, y1,
                                                      test_size=split,
                                                      random_state=1)
# for y2: Splitting the data into training and test sets (same splits)
_ , _ , y_train2, y_test2 = train_test_split(X, y2,
                                             test_size=split,
                                             random_state=1)

## Choice of k, Episode 1.
##############################################

In [ ]:
# (for adaptive choice of k_train, see below)
# We suggest the following rule of thumb:
# Select k_train as the largest k such that a spearman test  of independence
# between the k largest radii and the target  cannot reject the null.
# At the prediction test, theoretical guarantees cover any k_pred < k_train. 

ratios = np.linspace(50/ntrain, 0.2, num=50)
test_indep = mlx.test_indep_target_radius(X_train, y_train1, ratios,  norm_func)
# plot 
mlx.plot_indep_target_radius(test_indep)
# k ~ 1500 maximum on this example.

pvals = test_indep['pvalues']
i_max = np.max(np.where(pvals>0.05)[0])
k_max = (ratios[i_max]*ntrain).astype(int)
ratio_max = ratios[i_max]
print(k_max, mlx.round_signif(ratio_max, 2))
# indeed. 

# NB: one may check that results are identical with y2: the spearman test only
# relies on ranks and y2 is a non-decreasing transform of y1. 


# Set training and prediction ratios accordingly: 
ratio_train = 0.08
ratio_test = 0.02   # higher quantile than training quantile 
norm_X = norm_func(X)
thresh_predict = np.quantile(norm_X, 1-ratio_test)
k_train = ratio_train * ntrain

# Pick  a classifier model in sklearn, previously imported
model = GradientBoostingRegressor()
# Regressor class initialization
regressor1 = mlx.Regressor(model, norm_func) 
regressor2 = mlx.Regressor(model, norm_func)
 
# Training the models
thresh_train, _, X_train_extreme = regressor1.fit(X_train, y_train1, k=k_train)
regressor2.fit(X_train, y_train2, k=k_train)

# Prediction on the test data
y_pred_extreme1,  X_test_extreme, mask_test = regressor1.predict(
                                                X_test, thresh_predict)
y_pred_extreme2, _, _  = regressor2.predict(X_test, thresh_predict)

# Evaluation of Mean Squared Error (MSE)
y_test_extreme1 = y_test1[mask_test]  # Filter test labels based on mask
y_test_extreme2 = y_test2[mask_test]  # Filter test labels based on mask
MSE1 = mean_squared_error(y_test_extreme1, y_pred_extreme1)
MSE2 = mean_squared_error(y_test_extreme2, y_pred_extreme2)
print(f'MSE1: {MSE1:.4f}')
print(f'MSE2: {MSE2:.4f}')

# Display regression results
regressor1.plot_predictions(y_test_extreme1, y_pred_extreme1)
regressor2.plot_predictions(y_test_extreme2, y_pred_extreme2)
# prediction of y2 is more efficient.

In [ ]:
# ## How about performance in terms of the original target?
# back to original scale:

In [ ]:
z_test_extreme = mlx.inv_transform_target_lin(y_test_extreme1, X_test_extreme,
                                              norm_func)
z_pred_extreme1 = mlx.inv_transform_target_lin(y_pred_extreme1, X_test_extreme,
                                               norm_func)
z_pred_extreme2 = mlx.inv_transform_target_nonlin(y_pred_extreme2,
                                                  X_test_extreme,
                                                  norm_order)
# Compare with naive method: train model on the full dataset,
# predict on extreme covariates.
naive = model
naive.fit(X_train, mlx.inv_transform_target_lin(y_train1, X_train, norm_func))
z_pred_extreme_naive = naive.predict(X_test_extreme)
MSE_naive = mean_squared_error(z_test_extreme, z_pred_extreme_naive)
print(f'MSE for naive sklearn model trained on full data: {MSE_naive:.4f}')
MSE_meth1 = mean_squared_error(z_test_extreme, z_pred_extreme1)
print(f'MSE original target with linear method 1: {MSE_meth1:.4f}')
MSE_meth2 = mean_squared_error(z_test_extreme, z_pred_extreme2)
print(f'MSE original target  with nonlinear method 2: {MSE_meth2:.4f}')
regressor1.plot_predictions(z_test_extreme, z_pred_extreme_naive)
regressor1.plot_predictions(z_test_extreme, z_pred_extreme1)
regressor2.plot_predictions(z_test_extreme, z_pred_extreme2)

# Clearly method 2 beats method 1 and naive method,  
# However naive and  method 1 are comparable .

why?
ang_pred = X_test_extreme[:, 0] / (norm_func(X_test_extreme))
plt.scatter(ang_pred, z_test_extreme, c='black', alpha=0.5, label='true z')
plt.scatter(ang_pred, z_pred_extreme1, c='blue', alpha=0.5, label = 'pred1')
plt.scatter(ang_pred, z_pred_extreme2, c='red', alpha=0.5, label='pred2')
plt.xlabel('angle X[0]/ || X||')
plt.ylabel(' target (original z) values')
plt.legend()
plt.show()
# bad predictions with method 1 for largest z values; 


## CHOICE OF K - Episode 2
# cross-validation choice of k, thresh_predict fixed

In [ ]:
# (re) setting the prediction threshold. 
ratio_test = 0.02
norm_X = norm_func(X)
thresh_predict = np.quantile(norm_X, 1-ratio_test)

# set the range of candidates training threshold, using X_train only. 
ratio_train = np.geomspace(0.02, 0.25, num=10)
k_train = (ntrain * ratio_train).astype(int)
regressor1 = mlx.Regressor(model, norm_func)
regressor2 = mlx.Regressor(model, norm_func)

# cross-validation on the training set. 
kscores1 = []
kscores_sd1 = []
kscores2 = []
kscores_sd2 = []

!time consuming
for k in k_train:
    mean_scores, sd_mean_scores, _ = regressor1.cross_validate(
        X_train, y_train1, k=k, thresh_predict=thresh_predict,
        scoring=mean_squared_error,
        random_state=k*7)
    kscores_sd1.append(sd_mean_scores)
    kscores1.append(mean_scores)
    mean_scores, sd_mean_scores, _ = regressor2.cross_validate(
        X_train, y_train2, k=k, thresh_predict=thresh_predict,
        scoring=mean_squared_error,
        random_state=k*7)
    kscores_sd2.append(sd_mean_scores)
    kscores2.append(mean_scores)

kscores1 = np.array(kscores1)
kscores_sd1 = np.array(kscores_sd1)
kscores2 = np.array(kscores2)
kscores_sd2 = np.array(kscores_sd2)

plt.plot(k_train, kscores1)
plt.fill_between(k_train, kscores1 + 1.64 * kscores_sd1,
                 kscores1 - 1.64 * kscores_sd1, color='blue', alpha=0.2)
plt.title("optimal k, linear method 1") 
plt.show()

plt.plot(k_train, kscores2)
plt.fill_between(k_train, kscores2 + 1.64 * kscores_sd2,
                 kscores2 - 1.64 * kscores_sd2, color='blue', alpha=0.2)
plt.title("optimal k, nonlinear method 2")
plt.show()


i_opt1 = np.argmin(kscores1)
k_opt1 = k_train[i_opt1]
print(f'optimal k linear transfom method 1: {k_opt1}')

i_opt2 = np.argmin(kscores2)
k_opt2 = k_train[i_opt2]
print(f'optimal k linear transfom method 2: {k_opt2}')


# method 1: retrain (on training set ) and evaluate (on test set)
# using  optimal k
regressor1 = mlx.Regressor(model, norm_func) 
regressor2 = mlx.Regressor(model, norm_func)
thresh_train1, _ , X_train_extreme1 = regressor1.fit(X_train, y_train1,
                                                     k=k_opt1)
y_pred_extreme1,  X_test_extreme,  mask_test = regressor1.predict(
                                            X_test, thresh_predict)
y_test_extreme1 = y_test1[mask_test]  # Filter test labels based on mask
MSE1 = mean_squared_error(y_test_extreme1, y_pred_extreme1)
print(f'MSE transformed scale linear method1 : {MSE1:.4f}')

# method  2: idem 
thresh_train2, _ , X_train_extreme2 = regressor2.fit(X_train, y_train2,
                                                     k=k_opt2)
y_pred_extreme2,  X_test_extreme,  mask_test = regressor2.predict(
                                            X_test, thresh_predict)
y_test_extreme2 = y_test2[mask_test]  # Filter test labels based on mask
MSE2 = mean_squared_error(y_test_extreme2, y_pred_extreme2)
print(f'MSE transformed scale nonlinear method2 : {MSE2:.4f}')

# 
regressor1.plot_predictions(y_test_extreme1, y_pred_extreme1)
# much better than with rule-of-thumb choice of k
regressor2.plot_predictions(y_test_extreme2, y_pred_extreme2)

# # back to original scale
z_test_extreme = mlx.inv_transform_target_lin(y_test_extreme1, X_test_extreme,
                                              norm_func)
z_pred_extreme1 = mlx.inv_transform_target_lin(y_pred_extreme1, X_test_extreme,
                                               norm_func)
z_pred_extreme2 = mlx.inv_transform_target_nonlin(y_pred_extreme2,
                                                  X_test_extreme,
                                                  norm_order)
                                           
MSE_meth1 = mean_squared_error(z_test_extreme, z_pred_extreme1)
print(f'MSE original target with linear method 1: {MSE_meth1:.4f}')
MSE_meth2 = mean_squared_error(z_test_extreme, z_pred_extreme2)
print(f'MSE original target  with nonlinear method 2: {MSE_meth2:.4f}')
print(f'(recall) MSE for naive sklearn model trained on full data: {MSE_naive:.4f}')

regressor1.plot_predictions(z_test_extreme, z_pred_extreme1)
regressor2.plot_predictions(z_test_extreme, z_pred_extreme2)
regressor1.plot_predictions(z_test_extreme, z_pred_extreme_naive)

# now the gap between method1 and 2 is not so significative.
# both largely outperform the naive method (by a factor about 4). 


